In [3]:
# pip install numpy==1.21

In [21]:
import trimesh
import pyvista as pv
import os
import numpy as np
import pickle
import csv
import variables
import pickle

In [5]:
# helper: save to csv
def save_to_csv(dic, csv_file_name, csv_columns):
    dict_data = dic
    csv_file = csv_file_name
    try:
        with open(csv_file, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for data in dict_data:
                writer.writerow(data)
    except IOError:
        print("I/O error")
        

def get_intersections(f1, f2):
    '''
    f1: pial;  f2: white
    '''
    s1 = pv.read(f1)
    s2 = pv.read(f2)
    intersection, s1_split, s2_split = s1.intersection(s2)
    return len(intersection.points), len(s1.points)

def run_one_model(modelname, filepairs):
    dic = []
    cnt = 0
    for f1,f2 in filepairs:
        cnt += 1
        c1,c2 = get_intersections(f1,f2)
        dic.append({'file':f1, 'intersection':c1,'mesh':c2})
        
    save_to_csv(dic, modelname+'2MeshCollision.csv', list(dic.keys()))
    return dic
    

In [6]:
def get_pairs(prepial, prewhite):
    if prepial[-1] != '/':
        prepial += '/'
    if prewhite[-1] != '/':
        prewhite += '/'
    return [[prepial + f1, prewhite + f2] for f1, f2 in zip(os.listdir(prepial),os.listdir(prewhite))]

In [7]:
import csv
f = 'deepcsrWtPlCollision.csv'
dcsr_csv = [0]
with open(f, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        dcsr_csv.append(int(row['0']))

In [8]:
code_pial = '/data/users2/llu13/output/cortexode/pred/lh_pial/'
code_white = '/data/users2/llu13/output/cortexode/pred/lh_white/'

cf_white = '/data/users2/llu13/output/corticalflow/cfpp/lh_white'
cf_pial = '/data/users2/llu13/output/corticalflow/cfpp/lh_pial'

dcsr_pial = '/data/users2/washbee/speedrun/outputdirs/deepcsr-output_dir-timing/checkpoints/test-set/lh_pial/'
dcsr_white = '/data/users2/washbee/speedrun/outputdirs/deepcsr-output_dir-timing/checkpoints/test-set/lh_white'

v2c_pial = '/data/users2/washbee/speedrun/Vox2Cortex_fork/experiments/hcp/test_template_42016_DATASET_NAME/meshes/lh_pial/epoch81/'
v2c_white = '/data/users2/washbee/speedrun/Vox2Cortex_fork/experiments/hcp/test_template_42016_DATASET_NAME/meshes/lh_white/epoch81/'

freesurfer_pial = '/data/users2/llu13/output/cortexode/truth/samples/converted/lh_pial'
freesurfer_white = '/data/users2/llu13/output/cortexode/truth/samples/converted/lh_white'

## vtk output to a file

In [9]:
# redirect vtk info to a log file
import vtk
output=vtk.vtkFileOutputWindow()
output.SetFileName("log.txt")
vtk.vtkOutputWindow().SetInstance(output)

### ignore warnings

In [10]:
import warnings
warnings.filterwarnings("ignore")

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
from vtkmodules.vtkCommonCore import vtkLogger
vtkLogger.SetStderrVerbosity(vtkLogger.VERBOSITY_OFF)

from IPython.display import Audio
sound_file = 'hey.m4a'
notify=Audio(sound_file, autoplay=True)

# Prepare data

In [11]:
files = [[code_pial, code_white],
         [cf_pial, cf_white],
         [dcsr_pial, dcsr_white], 
         [v2c_pial, v2c_white], 
         [freesurfer_pial, freesurfer_white]]
model_names = ['cortexode',
               'corticalflow',
               'deepcsr',
               'vox2cortex',
               'freesurfer'
              ]

In [12]:
# ----- select model index -------

def get_file_pairs(index):
    prepial, prewhite = files[index]
    mn = model_names[index]
    filepairs = get_pairs(prepial, prewhite)
    return filepairs

# batch process

### processing

In [13]:
def get_xyz_limits(s1,ratio_x0 = 0.23, ratio_y0 = 0.807, ratio_y1 = 0.205, ratio_z1 = 0.388, extend = 20):
    maxvalues = np.max(np.array(s1.points), axis = 0)
    minvalues = np.min(np.array(s1.points), axis = 0)
    ux,uy,uz = maxvalues
    lx,ly,lz = minvalues
    x0 = ux - ratio_x0 * (ux - lx)
    x1 = ux + extend
    y0 = uy - ratio_y0 * (uy - ly)
    y1 = uy - ratio_y1 * (uy - ly)
    z0 = -uz - extend
    z1 = uz - ratio_z1 * (uz - lz)
    bounds = [x0,x1,y0,y1,z0,z1]
    return bounds

def removecells(s1,bounds):
    remove_idx = list(s1.find_cells_within_bounds(bounds))
    return s1.remove_cells(remove_idx)

def calc_after_remove_ratio(f1, f2):
    """
    for m1, m2: removing triangles
    for m3: removing lines
    """
    s1, s2 = pv.read(f1),pv.read(f2)
    d, _, _ = s1.intersection(s2); 
    bounds = get_xyz_limits(s1)
    m1 = removecells(s1,bounds)
    m2 = removecells(s2,bounds)
    m3 = removecells(d, bounds)
    if len(d.points):
        ratioPercent1 = (d.n_points) / (s1.n_cells + s2.n_cells) * 2 * 100  # not removing anything
    else:
        ratioPercent1 = 0
    if len(m3.points):
        ratioPercent2 = (m3.n_points) / (m1.n_cells + m2.n_cells) * 2 * 100  # after removing medial wall
        tmp = m3.n_points
    else:
        tmp = 0
        ratioPercent2 = 0
    
    return tmp, m1.n_cells, m2.n_cells, ratioPercent1, ratioPercent2

# 1 mesh collision calc

       calculate collision with `collision.pickle` file and original `.stl` file. 
       - calculate the boundaries with pre-designed ratios.
       - remove the area within medial wall with boundaries.
       - calculate the new collision rate.

    Steps:
    1. function 1: 
        given a single .pickle file and a .stl file, calculate 2 collision numbers, with and without removing the boundaries.
        note: the collision should be Percent. *100
        return .pickle file name; .stl file name; collision ratio 1; collision ratio2.
    2. function 2:
        take a folder, given the results, combining the folder name, with the return from function 1.
    3. function 3:
        loop to run all the folders.
        with the results from funtion 2, save the results in csv file.

In [92]:
def get_xyz_limits(s1,ratio_x0 = 0.23, ratio_y0 = 0.807, ratio_y1 = 0.205, ratio_z1 = 0.388, extend = 20):
    maxvalues = np.max(np.array(s1.points), axis = 0)
    minvalues = np.min(np.array(s1.points), axis = 0)
    ux,uy,uz = maxvalues
    lx,ly,lz = minvalues
    x0 = ux - ratio_x0 * (ux - lx)
    x1 = ux + extend
    y0 = uy - ratio_y0 * (uy - ly)
    y1 = uy - ratio_y1 * (uy - ly)
    z0 = -uz - extend
    z1 = uz - ratio_z1 * (uz - lz)
    bounds = [x0,x1,y0,y1,z0,z1]
    return bounds

def removecells(s1,bounds):
    remove_idx = list(s1.find_cells_within_bounds(bounds))
    return set(remove_idx), s1.remove_cells(remove_idx)

def removecells_self_intersection(d, remove_idx):
    res = 0
    for idx1, idx2 in d:
        if (idx1 not in remove_idx) and (idx2 not in remove_idx):
            res += 1
    return res

def function1(picklefile, stlfile):
    s1 = pv.read(stlfile)
    with open(picklefile, 'rb') as f:
        d = pickle.load(f)
#     print('stl file triangles: ', s1.n_cells)
    bounds = get_xyz_limits(s1)
    
    remove_idx, m1 = removecells(s1,bounds)
#     print('stl file after removal triangles: ', m1.n_cells)
    remain_len = removecells_self_intersection(d, remove_idx)
    
    ratioPercent1 = len(d) / s1.n_cells * 100
    ratioPercent2 = remain_len / m1.n_cells * 100
#     print(ratioPercent1, ratioPercent2)
    return ratioPercent1, ratioPercent2
    

In [91]:
def freesurfer_files_pial():
    res = []
    pre_stl = variables.freesurfer['lh_pial']
    pre_pickle = '../collision_data_csv/allCollisions/freesurfer/lh_pial/'
    files = os.listdir(pre_pickle)

    len_postfix = '-collisions.pkl'
    for f in files:
        _, f1, _ = f.split('-')
        file_stl_path = pre_stl + f1
        file_pkl_path = pre_pickle + f

        res.append([file_pkl_path, file_stl_path])
    return res

def freesurfer_files_white():
    res = []
    pre_stl = variables.freesurfer['lh_white']
    pre_pickle = '../collision_data_csv/allCollisions/freesurfer/lh_white/'
    files = os.listdir(pre_pickle)

    len_postfix = '-collisions.pkl'
    for f in files:
        _, f1, _ = f.split('-')
        
        file_stl_path = pre_stl + f1
        file_pkl_path = pre_pickle + f

        res.append([file_pkl_path, file_stl_path])
    return res

In [95]:
res = freesurfer_files_pial()


In [97]:
s1 = pv.read(res[0][1])
with open(res[0][0], 'rb') as f:
    d = pickle.load(f)

In [137]:
x = trimesh.load(res[0][1])

In [138]:
dir(x)

['__abstractmethods__',
 '__add__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_cache',
 '_center_mass',
 '_data',
 '_density',
 '_kwargs',
 '_visual',
 'apply_obb',
 'apply_scale',
 'apply_transform',
 'apply_translation',
 'area',
 'area_faces',
 'as_open3d',
 'body_count',
 'bounding_box',
 'bounding_box_oriented',
 'bounding_cylinder',
 'bounding_primitive',
 'bounding_sphere',
 'bounds',
 'center_mass',
 'centroid',
 'compute_stable_poses',
 'contains',
 'convert_units',
 'convex_decomposition',
 'convex_hull',
 'copy',
 'crc',
 'density',
 'difference',
 'edges',
 'edges_face',
 'edges_s

In [104]:
d

array([[125179, 127470],
       [125178, 127470],
       [ 97562, 101923],
       ...,
       [  5660,   5688],
       [170785, 172906],
       [188188, 188192]])

In [120]:
s1.cell[125179]

Cell (0x7f61117f0040)
  Type:        <CellType.TRIANGLE: 5>
  Linear:      True
  Dimension:   2
  N Points:    3
  N Faces:     0
  N Edges:     3
  X Bounds:    -4.410e+01, -4.376e+01
  Y Bounds:    -5.014e+00, -4.785e+00
  Z Bounds:    -2.531e+01, -2.477e+01

In [121]:
s1.cell[127470]

Cell (0x7f611164b9a0)
  Type:        <CellType.TRIANGLE: 5>
  Linear:      True
  Dimension:   2
  N Points:    3
  N Faces:     0
  N Edges:     3
  X Bounds:    -4.403e+01, -4.339e+01
  Y Bounds:    -5.311e+00, -4.820e+00
  Z Bounds:    -2.509e+01, -2.436e+01

In [ ]:
x: -44.1, -43.76   -44.03
y: -5.01, -4.785
z: -2.531, -2.477

In [127]:
s1.cell[257072]

IndexError: list index out of range

In [129]:
s1.n_cells

257072

In [131]:
tmp1 = set([i for i in range(s1.n_cells)])
tmp1.remove(125179)
tmp_mesh1 = s1.remove_cells(list(tmp1))

In [132]:
tmp1 = set([i for i in range(s1.n_cells)])
tmp1.remove(127470)
tmp_mesh2 = s1.remove_cells(list(tmp1))

In [134]:
d, _, _ = tmp_mesh1.intersection(tmp_mesh2)

ERROR:root:No points to subdivide
2023-05-19 20:39:56.676 (3632.792s) [        7EF55740]    vtkPointLocator.cxx:868    ERR| vtkPointLocator (0x2313340): No points to subdivide
2023-05-19 20:39:56.679 (3632.794s) [        7EF55740]vtkIntersectionPolyData:2410  WARN| No Intersection between objects 


In [135]:
d

PolyData,Information
N Cells,0
N Points,0
N Strips,0
X Bounds,"1.000e+299, -1.000e+299"
Y Bounds,"1.000e+299, -1.000e+299"
Z Bounds,"1.000e+299, -1.000e+299"
N Arrays,0


In [103]:
np.max(d)

256567

## all models

In [89]:
dic_ratios = {}
variables.models

['freesurfer',
 'cortexode',
 'corticalflow',
 'deepcsr',
 'vox2cortex',
 'pialnn',
 'topofit']

In [85]:
model = variables.models[0]
model

'freesurfer'

In [86]:
files_white = freesurfer_files_white()
files_pial = freesurfer_files_pial()

In [79]:
def get_all(pkl_stl_files):
    ratios = []
    for i,j in pkl_stl_files:
        ratios.append(function1(i,j))
    return ratios

In [93]:
dic_ratios[model] = {'lh_pial':get_all(files_pial),
                     'lh_white':get_all(files_white)
                    }

In [71]:
with open(res[100][0], 'rb') as f:
    d = pickle.load(f)

In [72]:
d

array([], shape=(0, 2), dtype=int64)

In [45]:
files

['0-200008_lh.pial_converted.stl-collisions.pkl',
 '100-290136_lh.pial_converted.stl-collisions.pkl',
 '101-293748_lh.pial_converted.stl-collisions.pkl',
 '102-295146_lh.pial_converted.stl-collisions.pkl',
 '103-297655_lh.pial_converted.stl-collisions.pkl',
 '104-298051_lh.pial_converted.stl-collisions.pkl',
 '105-298455_lh.pial_converted.stl-collisions.pkl',
 '106-299154_lh.pial_converted.stl-collisions.pkl',
 '11-202113_lh.pial_converted.stl-collisions.pkl',
 '1-200109_lh.pial_converted.stl-collisions.pkl',
 '12-202719_lh.pial_converted.stl-collisions.pkl',
 '13-203418_lh.pial_converted.stl-collisions.pkl',
 '14-203721_lh.pial_converted.stl-collisions.pkl',
 '15-203923_lh.pial_converted.stl-collisions.pkl',
 '16-204016_lh.pial_converted.stl-collisions.pkl',
 '17-204319_lh.pial_converted.stl-collisions.pkl',
 '18-204420_lh.pial_converted.stl-collisions.pkl',
 '20-204622_lh.pial_converted.stl-collisions.pkl',
 '21-205119_lh.pial_converted.stl-collisions.pkl',
 '2-200210_lh.pial_convert

In [ ]:
r

In [41]:
files[-1]

'lh.pial_converted.stl'

In [5]:
import os
os.listdir('../collision_data_csv/allCollisions/')

['freesurfer',
 '.ipynb_checkpoints',
 'cortexode',
 'corticalflow',
 'deepcsr',
 'pialnn',
 'topofit',
 'v2c']

In [16]:
f1 = '../collision_data_csv/allCollisions/freesurfer/lh_pial/1-200109_lh.pial_converted.stl-collisions.pkl'


'/data/users2/llu13/output/cortexode/truth/samples/converted/lh_pial/'

In [23]:
pre = '/data/users2/llu13/output/cortexode/truth/samples/converted/lh_pial/'
f2 = os.listdir(pre)[1]

In [28]:
function1(f1, pre + f2)

stl file triangles:  280218
stl file after removal triangles:  256801
1266 1194


(0.4517911054964349, 0.46495146046939073)

In [31]:
1266/280218

0.004517911054964349

In [12]:
with open(f1, 'rb') as f:
    c = pickle.load(f)

In [14]:
c[0]

array([80710, 82705])

In [ ]:
def function1()

# 2 mesh collision calc

In [14]:
results = []
csvfile = 'Intersection.csv'

 Calculate

In [22]:
index = 4
filepairs = get_file_pairs(index)
mn = model_names[index]
mn

'freesurfer'

In [23]:
start = 0
end = start + 30

with open("Intersection.csv", "a") as csvfile:
    writer = csv.writer(csvfile)
    for i,(f1,f2) in enumerate(filepairs[start:]):
        real_idx = i + start
        print(real_idx)
        if index == 2:
            if real_idx in dcsr_csv:
                m3,m1,m2,ratioPercent1,ratioPercent2 = calc_after_remove_ratio(f1, f2)
                row = [mn, real_idx, ratioPercent1, ratioPercent2, f1]
                results.append(row)
                writer.writerow(row)
        else:
            m3,m1,m2,ratioPercent1,ratioPercent2 = calc_after_remove_ratio(f1, f2)
            row = [mn, real_idx, ratioPercent1, ratioPercent2, f1]
            results.append(row)
            writer.writerow(row)
        
notify

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107


In [24]:
f1, f2 = filepairs[1]

In [25]:
s1, s2 = pv.read(f1),pv.read(f2)

In [26]:
d, _, _ = s1.intersection(s2); 

In [28]:
d.shape

AttributeError: 'PolyData' object has no attribute 'shape'

In [27]:
d.n_points

4519

In [18]:
bounds = get_xyz_limits(s1)
m1 = removecells(s1,bounds)
m2 = removecells(s2,bounds)
m3 = removecells(d, bounds)

In [21]:
m3.n_points

187

In [ ]:
m3,m1,m2,ratioPercent1,ratioPercent2 = calc_after_remove_ratio(f1, f2)

In [1]:
results

NameError: name 'results' is not defined

In [29]:
from IPython.display import Audio
sound_file = 'hey.m4a'
notify=Audio(sound_file, autoplay=True)

# -------your code----------
# code that run long long long time
# code that run long long long time
# code that run long long long time
# code that run long long long time
# code that run long long long time
# code that run long long long time
# code that run long long long time
# code that run long long long time
# code that run long long long time
# code that run long long long time
# code that run long long long time

notify

In [1]:
start

NameError: name 'start' is not defined

60

In [74]:
len(

[['cortexode',
  0,
  3.3981876332622605,
  1.5923942689837132,
  '/data/users2/llu13/output/cortexode/pred/lh_pial/adni_lh_200008.pial_converted.stl'],
 ['cortexode',
  1,
  3.9358887124820443,
  1.7991047654687027,
  '/data/users2/llu13/output/cortexode/pred/lh_pial/adni_lh_200109.pial_converted.stl'],
 ['cortexode',
  2,
  2.3206830642321816,
  0.9300864529698576,
  '/data/users2/llu13/output/cortexode/pred/lh_pial/adni_lh_200210.pial_converted.stl'],
 ['cortexode',
  0,
  3.3981876332622605,
  1.5923942689837132,
  '/data/users2/llu13/output/cortexode/pred/lh_pial/adni_lh_200008.pial_converted.stl'],
 ['cortexode',
  1,
  3.9358887124820443,
  1.7991047654687027,
  '/data/users2/llu13/output/cortexode/pred/lh_pial/adni_lh_200109.pial_converted.stl'],
 ['cortexode',
  2,
  2.3206830642321816,
  0.9300864529698576,
  '/data/users2/llu13/output/cortexode/pred/lh_pial/adni_lh_200210.pial_converted.stl'],
 ['cortexode',
  3,
  3.3994432846207374,
  1.415855739519439,
  '/data/users2/llu

In [58]:
results[0].n_points

3576

In [27]:
m1.points

pyvista_ndarray([[-29.464205  ,  11.062517  , -62.205467  ],
                 [-30.23411   ,  11.516321  , -62.789597  ],
                 [-28.742605  ,  11.5973    , -62.61608   ],
                 ...,
                 [ -9.109061  ,  -0.45121315,  48.731777  ],
                 [ -8.293487  ,  -0.7458459 ,  48.619736  ],
                 [ -7.108672  ,  -1.1649586 ,  48.384357  ]],
                dtype=float32)

In [26]:
m2.points

pyvista_ndarray([[-30.999702 ,  12.80227  , -57.85451  ],
                 [-31.502422 ,  13.272774 , -57.89746  ],
                 [-30.540493 ,  13.407418 , -58.065712 ],
                 ...,
                 [ -8.927928 ,  -1.1509303,  46.727367 ],
                 [ -8.337483 ,  -1.2089436,  47.034237 ],
                 [ -7.4900618,  -1.4409716,  46.89572  ]], dtype=float32)

In [12]:
f1,f2 = filepairs[0]

In [28]:
intersection_rate = 

3576

In [23]:
m3. 

PolyData,Information
N Cells,224131
N Points,112519
N Strips,0
X Bounds,"-6.058e+01, 4.911e+00"
Y Bounds,"-8.309e+01, 8.606e+01"
Z Bounds,"-6.340e+01, 5.016e+01"
N Arrays,0


In [24]:
m2

PolyData,Information
N Cells,225004
N Points,112970
N Strips,0
X Bounds,"-5.756e+01, 3.300e+00"
Y Bounds,"-8.162e+01, 8.278e+01"
Z Bounds,"-5.884e+01, 4.715e+01"
N Arrays,0


In [25]:
m3

PolyData (0x7f5d4f31cc40)
  N Cells:    3568
  N Points:   3576
  N Strips:   0
  X Bounds:   -4.692e+01, 2.118e+00
  Y Bounds:   -7.521e+01, 1.520e+01
  Z Bounds:   -4.600e+01, 3.476e+01
  N Arrays:   3

In [22]:
m3.n_points

3576

In [18]:
m3. (m1.n_cells + m2.n_cells)

224131

In [14]:
dir(m1)

['ALL_PIECES_EXTENT',
 'AddCellReference',
 'AddObserver',
 'AddReferenceToCell',
 'Allocate',
 'AllocateCellGhostArray',
 'AllocateCopy',
 'AllocateEstimate',
 'AllocateExact',
 'AllocatePointGhostArray',
 'AllocateProportional',
 'AttributeTypes',
 'BOUNDING_BOX',
 'BreakOnError',
 'BuildCellLocator',
 'BuildCells',
 'BuildLinks',
 'BuildLocator',
 'BuildPointLocator',
 'CELL',
 'CELL_DATA_FIELD',
 'CELL_DATA_VECTOR',
 'CheckAttributes',
 'ComputeBounds',
 'ComputeCellsBounds',
 'CopyAttributes',
 'CopyCells',
 'CopyInformationFromPipeline',
 'CopyInformationToPipeline',
 'CopyStructure',
 'Crop',
 'DATA_EXTENT',
 'DATA_EXTENT_TYPE',
 'DATA_NUMBER_OF_GHOST_LEVELS',
 'DATA_NUMBER_OF_PIECES',
 'DATA_OBJECT',
 'DATA_OBJECT_FIELD',
 'DATA_PIECE_NUMBER',
 'DATA_TIME_STEP',
 'DATA_TYPE_NAME',
 'DIRECTION',
 'DataHasBeenGenerated',
 'DebugOff',
 'DebugOn',
 'DeepCopy',
 'DeleteCell',
 'DeleteCells',
 'DeleteLinks',
 'DeletePoint',
 'EDGE',
 'EDGE_DATA_VECTOR',
 'ERR_INCORRECT_FIELD',
 'ERR_

In [17]:
m2

PolyData,Information
N Cells,225004
N Points,112970
N Strips,0
X Bounds,"-5.756e+01, 3.300e+00"
Y Bounds,"-8.162e+01, 8.278e+01"
Z Bounds,"-5.884e+01, 4.715e+01"
N Arrays,0


In [18]:
m3

PolyData (0x7f1a889b6ee0)
  N Cells:    3568
  N Points:   3576
  N Strips:   0
  X Bounds:   -4.692e+01, 2.118e+00
  Y Bounds:   -7.521e+01, 1.520e+01
  Z Bounds:   -4.600e+01, 3.476e+01
  N Arrays:   3

In [9]:
start = 0
end = start + 1

a = []
for i,(f1,f2) in enumerate(filepairs[start: ]):
    tmp_idx = i + start
    s1,s2 = pv.read(f1),pv.read(f2)
    d, _, _ = s1.intersection(s2); 
    # ----------------------calc and cut----------------------
    bounds = get_xyz_limits(s1)
    m1 = removecells(s1,bounds)
    m2 = removecells(s2,bounds)
    m3 = removecells(intersection, bounds)
    
    a.append([f1, tmp_idx, len(d.points),len(e.points), len(m1.)])

# do DCSR

In [10]:
f1,f2 = filepairs[0]

In [59]:
f1 = '/data/users2/washbee/speedrun/outputdirs/deepcsr-output_dir-timing/checkpoints/test-set/lh_pial/201818_lh_pial.stl'

In [60]:
f2 = '/data/users2/washbee/speedrun/outputdirs/deepcsr-output_dir-timing/checkpoints/test-set/lh_white/201818_lh_white.stl'

In [61]:
f1

'/data/users2/washbee/speedrun/outputdirs/deepcsr-output_dir-timing/checkpoints/test-set/lh_pial/201818_lh_pial.stl'

In [62]:
f2

'/data/users2/washbee/speedrun/outputdirs/deepcsr-output_dir-timing/checkpoints/test-set/lh_white/201818_lh_white.stl'

In [63]:
s1 = pv.read(f1)
s2 = pv.read(f2)

In [14]:
# pl = pv.Plotter(window_size=(800,800))
# _ = pl.add_mesh(s1, style='wireframe', color='blue', line_width=10)

# _ = pl.add_mesh(s2, style='wireframe',color='red', line_width=10)
# _ = pl.add_mesh(intersection, color=c[2], line_width=line_width2)
# pl.view_yz()
# pl.set_viewup(view_up1)


# pl.show()

In [64]:
intersection, s1_split, s2_split = s1.intersection(s2)

# pickle save intersection.points, s1.points, s2.points

## get min, max for x, y, z in the brain

In [65]:
fname = f1.split('/')[-1].split('.')[0] + '_white'
with open('./dcsr-intersection-{}.pickle'.format(fname),'wb') as p:
    pickle.dump(intersection, p)

In [67]:
dic = {'max-s1':np.array(np.max(s1.points,axis=0)),
       'min-s1':np.array(np.min(s1.points,axis=0)),
       'max-s2':np.array(np.max(s2.points,axis=0)),
       'min-s2':np.array(np.min(s2.points,axis=0)),
      }
with open('./dcsr-s1s2-{}.pickle'.format(fname),'wb') as p:
    pickle.dump(dic, p)

In [54]:
np.max(s1.points,axis=0)

pyvista_ndarray([ 3.7536075, 72.08017  , 80.0375   ], dtype=float32)

In [57]:
np.min(s1.points,axis=0)

pyvista_ndarray([ -69.73621 , -108.386856,  -48.4087  ], dtype=float32)

In [58]:
f1

'/data/users2/washbee/speedrun/outputdirs/deepcsr-output_dir-timing/checkpoints/test-set/lh_pial/200008_lh_pial.stl'

In [34]:
import pickle

with open('./intersection-all-test.pickle','wb') as p:
    pickle.dump(intersection, p)

In [35]:
with open('./intersection-all-test.pickle','rb') as p:
    hehe = pickle.load(p)

In [ ]:
for f1,f2 in filepairs[:5]:
    s1 = pv.read(f1)
    s2 = pv.read(f2)
    s1.save('s1.vtk')
    s2.save('s2.vtk')
    s1 = pv.read('s1.vtk')
    s2 = pv.read('s2.vtk')
    try:
        d, t, _ = s1.intersection(s2)
        print(len(d.points), len(t.points))
    except:
        print(f1)
    

In [15]:
f1

'/data/users2/washbee/speedrun/outputdirs/deepcsr-output_dir-timing/checkpoints/test-set/lh_pial/200008_lh_pial.stl'

In [14]:
s1.intersection(s2)

(PolyData (0x7f46849e19a0)
   N Cells:    1925
   N Points:   1878
   N Strips:   0
   X Bounds:   -2.725e+01, 2.109e-01
   Y Bounds:   -9.440e+01, 3.099e+01
   Z Bounds:   -2.063e+01, 2.799e+01
   N Arrays:   3,
 PolyData (0x7f46849e1ac0)
   N Cells:    152309
   N Points:   154858
   N Strips:   0
   X Bounds:   -6.974e+01, 3.754e+00
   Y Bounds:   -1.084e+02, 7.208e+01
   Z Bounds:   -4.841e+01, 8.004e+01
   N Arrays:   0,
 PolyData (0x7f46849e18e0)
   N Cells:    0
   N Points:   0
   N Strips:   0
   X Bounds:   1.000e+299, -1.000e+299
   Y Bounds:   1.000e+299, -1.000e+299
   Z Bounds:   1.000e+299, -1.000e+299
   N Arrays:   0)

In [9]:
s = s1
points = s.points
faces = s.faces
mesh = pv.PolyData(points, faces)
mesh1 = mesh
repeat = points[0]
lastrepeat = points[-1]

s = s2
points = s.points[1:-1]
faces = s.faces
mesh = pv.PolyData([repeat] + points + [lastrepeat], faces)
mesh2 = mesh

In [ ]:
d, _, _ = mesh1.intersection(mesh2); 

In [ ]:
len(d.points)

In [ ]:
len(s1.points)

In [ ]:
len(d.points),len(e.points)

In [ ]:
del s1,s2,d,e,c

In [ ]:
#     s1,s2 = pv.read(f1),pv.read(f2)
#     d, e, c = s1.intersection(s2); 
#     a.append([len(d.points),len(e.points)])

In [ ]:
import time

In [ ]:
for f1,f2 in filepairs[start: ]:
    s1 = pv.read(f1)
    s2 = pv.read(f2)
    d, e, c = s1.intersection(s2); 
    del s1,s2,f1,f2
    a.append([len(d.points),len(e.points)])
    time.sleep(10)
    del d,e,c

In [ ]:
start = 0
end = start + 5

a = []
for f1,f2 in filepairs[start: ]:
    s1 = pv.read(f1)
    s2 = pv.read(f2)
    d, e, c = s1.intersection(s2); 
    del s1,s2,f1,f2
    a.append([len(d.points),len(e.points)])
    time.sleep(10)
    del d,e,c

## using Pymesh

In [ ]:
!pip install pymesh

In [ ]:
import pymesh

In [ ]:
s1.faces

In [ ]:
s1.verts

In [ ]:
s1.verts

In [ ]:
pymesh.load_mesh(f1)

## using Trimesh

In [ ]:
mesh = trimesh.load_mesh(f1)

In [ ]:
with open(f1,'rb') as f_obj:
    mesh = trimesh.load_mesh(f_obj)

## get file tytpe using mimetype

In [ ]:
import mimetypes

mime_type, _ = mimetypes.guess_type(f1)

print(mime_type)


## using pyvista

In [ ]:
s = s1
points = s.points
faces = s.faces
mesh = pv.PolyData(points, faces)
mesh1 = mesh

In [ ]:
s = s2
points = s.points
faces = s.faces
mesh = pv.PolyData(points, faces)
mesh2 = mesh

In [ ]:
help(mesh1.intersection)

In [ ]:
mesh1.intersection(mesh2, tolerance = .1)

## set tolerance

In [ ]:
filter = pv.Intersect(polydataA=mesh1, polydataB=mesh2)
filter.set(abs_tol=0.1, rel_tol=0.01)
intersection = filter.run()

In [ ]:
try
intersection = mesh1.extract_surface().boolean_difference(mesh2, tolerance=0.1)

## original method

In [ ]:
s1.intersection(s2)

In [ ]:
trimesh.intersections()

In [ ]:
points

In [ ]:
faces

In [ ]:
!pip install meshpy

In [ ]:
try:
    
except Exception:
    sys.exc_clear()
    

In [ ]:
import meshpy.triangle as triangle

intersection = triangle.intersection(mesh1, mesh2)

# Print the resulting mesh
print(intersection)


In [ ]:
# Get the VTK data object
vtk_data = s1.GetOutput()

# Cast to a vtkPolyData object
polydata_vtk = vtk.vtkPolyData.SafeDownCast(vtk_data)

In [ ]:
s1.intersection(s2)

## try using numpy_stl

In [ ]:
!pip install stl

In [ ]:
!pip install numpy-stl --upgrade

In [ ]:
!pip install stl

In [ ]:
type(s1)

In [ ]:
dir(s1)

In [ ]:
from stl import mesh

with open(f1, 'rb') as f:
    mesh = mesh.Mesh.from_bytes(stl_data)
#     mesh1 = trimesh.exchange.stl.load_stl(f.read())

In [ ]:
dir(s1)

In [ ]:
# mesh1 = trimesh.exchange.stl.load_stl_binary(s1)
mesh1 = trimesh.exchange.stl.load_stl_binary(s1)

In [ ]:
mesh1 = trimesh.load(f1)
mesh2 = trimesh.load(f2)

In [ ]:
import trimesh

# Load the meshes


# Calculate the intersection points
intersection = trimesh.intersections.mesh_multiplane(mesh1, mesh2)

# Print the intersection points
print(intersection)


# testing pyvista intersection function

In [ ]:
s1 = pv.read(f1)
s2 = pv.read(f2)

In [ ]:
# result = s1.boolean_intersection(s2) # takes really long time to run

In [ ]:
s1.intersection(s2)

In [ ]:
reader = pv.get_reader(f1)
mesh1 = reader.read()
reader = pv.get_reader(f2)
mesh2 = reader.read()

In [ ]:
!pip install vtki

In [ ]:
import vtk
import vtki

In [ ]:
f1

In [ ]:

def get_mesh_intersection(mesh1, mesh2):
    '''
    Find the intersection volume between mesh vtk objects
    and return the volume of intersection
    '''
    alg = vtk.vtkBooleanOperationPolyDataFilter()
    alg.SetInputData(0, mesh2)
    alg.SetInputData(1, mesh1)
    alg.SetOperationToIntersection()
    alg.Update()
    intersection = vtki.wrap(alg.GetOutput())
    return intersection

# Load meshes
# Load meshes
mesh_i = vtki.read(f1)
mesh_j = vtki.read(f2)

print('VOLUME of probe1:\t', mesh_i.volume, '# of Polys:\t', mesh_i.n_faces)
print('VOLUME of probe2:\t', mesh_j.volume, '# of Polys:\t', mesh_j.n_faces)
intersection = get_mesh_intersection(mesh_i, mesh_j)
print('INTERSECTION VOLUME:\t', intersection.volume)

print('VOLUME of probe1:\t', mesh1.volume, '# of Polys:\t', mesh1.n_faces)
print('VOLUME of probe2:\t', mesh2.volume, '# of Polys:\t', mesh2.n_faces)
intersection = get_mesh_intersection(mesh1, mesh2)
print('INTERSECTION VOLUME:\t', intersection.volume)

In [ ]:
s1.intersection(s2)

## vtk reader stl

In [ ]:
import vtk
file = f1
# Create a renderer and window
renderer = vtk.vtkRenderer()
render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)

# Create an STL reader and set the input file
reader = vtk.vtkSTLReader()
reader.SetFileName(file)

# Create a mapper and actor to display the geometry
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(reader.GetOutputPort())
actor = vtk.vtkActor()
actor.SetMapper(mapper)
renderer.AddActor(actor)

In [ ]:
reader

In [ ]:
dir(s1)

In [ ]:
help(s1.intersection)

In [ ]:
s1.vtk

In [ ]:
dir(s1)

In [ ]:
help(s1)

In [ ]:

intersection, s1_split, s2_split = s1.intersection(s2)
return len(intersection.points), len(s1.points)

# run code

In [ ]:
dic = run_one_model('cortexode', filepairs[:3])

In [ ]:
prepial, prewhite = cf_pial, cf_white
filepairs = get_pairs(prepial, prewhite)
run_one_model('cf', filepairs)

In [ ]:
prepial, prewhite = dcsr_pial, dcsr_white
filepairs = get_pairs(prepial, prewhite)
run_one_model('dcsr', filepairs)

In [ ]:
prepial, prewhite = v2c_pial, v2c_white
filepairs = get_pairs(prepial, prewhite)
run_one_model('v2c', filepairs)